In [20]:
import os
import pandas as pd
import numpy as np
import scipy
import pickle
import sys  

In [19]:
# Location of the data folder
%store -r dataDir

# Pathology INDDID

In [21]:
# Load new_pathT: ex-vivo histopathology Data (Quantification) / %AO for pathology regions
new_pathT = pd.read_excel(os.path.join(dataDir, 'NewFTDData', 'FTLD Library 4-25-23 update.xlsx'), 
                          dtype={'INDDID': str, 'Tau1_TDP2': str})

### Format the Pathology Data - %AO to desired format

#### Divide each INDDID into {GM, WM} and {L, R} - 22 Regions (They are alphabetically Ordered)

In [22]:
# For each INDDID divided into {GM, WM} and {L, R} (maximum 4 rows per INDDID)
pathT_WMGM = pd.pivot_table(new_pathT, values='AvgPercentAO', 
                            index=['INDDID', 'FullAutopsyID', 'AutopsyIDNumOnly', 
                                   'Tau1_TDP2', 'Hemisphere_by_slide', 'AnalysisRegion'], 
                            columns=['Region'], aggfunc=np.sum)

In [23]:
# Unstacking the Index --> Need a way to solve this without saving to csv format
pathT_WMGM.to_csv(os.path.join(dataDir, 'NewFTDData', 'new_pathT(GMWM).csv'))
pathT_WMGM = pd.read_csv(os.path.join(dataDir, 'NewFTDData', 'new_pathT(GMWM).csv'))

#### Divide the pathT into GM and WM 

In [24]:
pathT_WMGM_type = pathT_WMGM.groupby('AnalysisRegion')

# This contains 2 seperate rows for {L, R}
pathT_GM_LR = pathT_WMGM_type.get_group('GM')
pathT_WM_LR = pathT_WMGM_type.get_group('WM')

# Combine 2 Rows for {L, R} into a single row
pathT_GM_LR_type = pathT_GM_LR.groupby('Hemisphere_by_slide')
pathT_GM_L = pathT_GM_LR_type.get_group('L')
pathT_GM_R = pathT_GM_LR_type.get_group('R')
pathT_GM = pd.merge(pathT_GM_L, pathT_GM_R, left_on=['INDDID', 'FullAutopsyID', 'AutopsyIDNumOnly', 'Tau1_TDP2', 'AnalysisRegion'], right_on=['INDDID', 'FullAutopsyID', 'AutopsyIDNumOnly', 'Tau1_TDP2', 'AnalysisRegion'], how='outer', suffixes=('_L', '_R')) 

pathT_WM_LR_type = pathT_WM_LR.groupby('Hemisphere_by_slide')
pathT_WM_L = pathT_WM_LR_type.get_group('L')
pathT_WM_R = pathT_WM_LR_type.get_group('R')
pathT_WM = pd.merge(pathT_WM_L, pathT_WM_R, left_on=['INDDID', 'FullAutopsyID', 'AutopsyIDNumOnly', 'Tau1_TDP2', 'AnalysisRegion'], right_on=['INDDID', 'FullAutopsyID', 'AutopsyIDNumOnly', 'Tau1_TDP2', 'AnalysisRegion'], how='outer', suffixes=('_L', '_R'))

# Drop Hemisphere_by_slide {L, R} Columns
pathT_GM = pathT_GM.drop(columns=['Hemisphere_by_slide_L', 'Hemisphere_by_slide_R'])
pathT_WM = pathT_WM.drop(columns=['Hemisphere_by_slide_L', 'Hemisphere_by_slide_R']) 

In [32]:
# Index for the case with tau or tdp for patients
FTD_TAUIndx = (pathT_GM.Tau1_TDP2 == 1)  # False or True
FTD_TDPIndx = (pathT_GM.Tau1_TDP2 == 2) # False or True

In [81]:
path_id_TAU = pathT_GM[FTD_TAUIndx]['INDDID'].values

In [82]:
path_id_TDP = pathT_GM[FTD_TDPIndx]['INDDID'].values

In [86]:
path_id_TAU

array([100551.  , 101068.  , 101105.  , 102149.  , 103032.  , 105000.  ,
       105223.  , 105358.  , 105492.  , 105961.  , 106297.  , 106309.  ,
       106814.  , 106840.  , 107516.  , 107663.  , 107969.  , 108026.  ,
       108077.  , 108196.  , 108508.  , 109115.  , 109176.  , 109299.  ,
       109759.  , 110181.  , 110306.  , 110745.  , 110914.  , 110917.  ,
       111231.  , 111527.  , 111530.  , 112570.  , 113113.  , 113909.  ,
       113938.  , 115001.  , 115592.  , 116504.  , 116591.  , 116607.  ,
       118011.  , 118410.  , 118780.  , 119113.  , 119140.  , 119359.  ,
       119413.  , 120298.  , 122143.  , 101407.  , 101483.  , 103121.  ,
       103782.  , 104281.  , 104937.  , 105564.  , 107187.  , 107429.  ,
       107667.  , 107677.  , 109048.  , 111005.  , 111853.  , 112514.  ,
       112764.  , 114348.  , 114762.  , 114762.02, 115327.  , 116275.  ,
       116401.  , 116409.  , 117566.  , 118575.  ])

In [84]:
path_id_TDP

array([100071.  , 100686.  , 102792.  , 103282.  , 103601.  , 103640.  ,
       103703.  , 103714.  , 104156.  , 104613.  , 104659.  , 104862.  ,
       105247.  , 105686.  , 105769.  , 106335.  , 106461.  , 106641.  ,
       107031.  , 108276.  , 108344.  , 108783.  , 108783.09, 109058.  ,
       109073.  , 109206.  , 109476.  , 110338.  , 110361.  , 110445.  ,
       110581.  , 110658.  , 110705.  , 111077.  , 112202.  , 112273.  ,
       112298.  , 112780.  , 112974.  , 113867.  , 114076.  , 114395.  ,
       114753.  , 116521.  , 116569.  , 116598.  , 116748.  , 117589.  ,
       117630.  , 117637.  , 117663.  , 117753.  , 118190.  , 118234.  ,
       118430.  , 118694.  , 118762.  , 118914.  , 118952.  , 119454.  ,
       119610.  , 119768.  , 120720.  , 120950.  , 121078.  , 121199.  ,
       121261.  , 100096.  , 101045.  , 101272.  , 101525.  , 101778.  ,
       103568.  , 104094.  , 106955.  , 107204.01, 107519.  , 107636.  ,
       108542.  , 108790.  , 108930.  , 109050.  , 

In [87]:
len(path_id_TAU)

76

In [88]:
len(path_id_TDP)

103

# MR INDDID

In [89]:
# MRI Thickness value for All Subjects - schaefer400x7
thicknessAllraw = pd.read_csv(os.path.join(dataDir, 'NewFTDData', 
                                           'invivoPathCohort_quantsSubSesSchaefer400_tian12.csv'), dtype={'id': str})

### Loading Look Up Table for Type of MRI Thickness Subjects

In [90]:
thicknessPathLUT = pd.read_excel(os.path.join(dataDir, 'NewFTDData', 'InvivoPathCohort_03172023.xls'), 
                                 dtype={'INDDID': str})

### Join the Above two dataframes on INDDID (Keep only the ones that INDDID are overlapping)

In [91]:
thicknessAll = pd.merge(thicknessAllraw, thicknessPathLUT, left_on='id', right_on='INDDID', how='inner') 

# We only lose INDDID 108783x09 in the thicknessAllraw (849 rows lost)

### Group by path type

In [92]:
thickness_path_type = thicknessAll.groupby('Group')

# MRI Thickness values for Healthy Control
thicknessHC = thickness_path_type.get_group('HC')
# MRI Thickness values for Patient (TAU)
thicknessPatientTAU = thickness_path_type.get_group('tau')
# MRI Thickness values for Patient (TDP)
thicknessPatientTDP = thickness_path_type.get_group('tdp')

In [102]:
# IDs
thick_id_HC = np.unique(thicknessHC.INDDID)
thick_id_TAU = np.unique(thicknessPatientTAU.INDDID)
thick_id_TDP = np.unique(thicknessPatientTDP.INDDID)

thick_id_HC = thick_id_HC.astype('float64')
thick_id_TAU = thick_id_TAU.astype('float64')
thick_id_TDP = thick_id_TDP.astype('float64')

In [103]:
thick_id_HC

array([100115., 100338., 100387., 100488., 100742., 101080., 102187.,
       102429., 102967., 102974., 103023., 103073., 103118., 103824.,
       104102., 105521., 105601., 106036., 106135., 106732., 106897.,
       107045., 107700., 108014., 108560., 108853., 108856., 109075.,
       109194., 109212., 109325., 109346., 109417., 109481., 110342.,
       111151., 111611., 112359., 112916., 113326., 115226., 115311.,
       115629., 116543., 117957., 117964., 118177., 118752., 118885.,
       119158., 119535., 119949., 120506., 124747.])

In [104]:
len(thick_id_HC)

54

In [105]:
len(thick_id_TAU)

26

In [106]:
len(thick_id_TDP)

30

# FTLD Autopsy MRI

In [64]:
ftd_autopsyT = pd.read_excel(os.path.join(dataDir, 'NewFTDData', 'FTLD Autopsy MRI (2023.02.10 14.31).xlsx'))

In [65]:
ftd_autopsyT.INDDID.values

array([122375., 123352., 122104., ..., 101778., 100518.,     nan])

In [66]:
autopsy_id = np.unique(ftd_autopsyT.INDDID)

In [108]:
autopsy_id

array([100013.  , 100518.  , 100551.  , 100906.  , 101045.  , 101407.  ,
       101483.  , 101525.  , 101778.  , 102049.  , 102792.  , 103782.  ,
       104093.  , 104094.  , 104190.  , 104613.  , 104862.  , 104937.  ,
       105223.  , 105247.  , 105564.  , 105769.  , 105961.  , 106309.  ,
       106814.  , 107077.  , 107516.  , 107519.  , 107677.  , 107890.  ,
       108026.  , 108276.  , 108344.  , 108508.  , 108542.  , 108783.  ,
       108783.09, 108790.  , 109013.  , 109050.  , 109058.  , 109073.  ,
       109176.  , 109198.  , 109300.  , 109476.  , 109757.  , 109759.  ,
       110005.  , 110445.  , 111231.  , 111862.  , 112222.  , 112273.  ,
       112780.  , 112974.  , 113113.  , 113909.  , 114395.  , 114753.  ,
       114762.  , 114978.  , 115001.  , 115188.  , 115264.  , 116283.  ,
       116504.  , 116569.  , 116591.  , 116607.  , 116828.  , 116831.  ,
       117468.  , 117515.  , 117566.  , 117589.  , 117630.  , 117637.  ,
       117774.  , 118011.  , 118064.  , 118148.  , 

## PATH

In [153]:
tau_aut_over = np.isin(path_id_TAU, autopsy_id)
print(f"Number of overlap (Path-Tau): {tau_aut_over.sum()}/{len(path_id_TAU)}")

Number of overlap (Path-Tau): 33/76


In [154]:
tdp_aut_over = np.isin(path_id_TDP, autopsy_id)
print(f"Number of overlap (Path-Tdp): {tdp_aut_over.sum()}/{len(path_id_TDP)}")

Number of overlap (Path-Tdp): 45/103


### BVFTD

In [188]:
ftd_autopsyT[ftd_autopsyT['INDDID'].isin(path_id_TAU[tau_aut_over])][['INDDID','BVFTD','ClinicalDx1']].drop_duplicates(subset=['INDDID','BVFTD','ClinicalDx1']).sort_values(by='INDDID')

,INDDID,BVFTD,ClinicalDx1
1083,100551.0,0.0,FTLD-PPA (PNFA)
717,101407.0,0.0,Corticobasal syndrome
472,101483.0,1.0,FTLD-bvFTD
1224,103782.0,0.0,Corticobasal syndrome
835,104937.0,0.0,Progressive supranuclear palsy
201,105223.0,1.0,FTLD-bvFTD
346,105564.0,1.0,FTLD-bvFTD
782,105961.0,0.0,Progressive supranuclear palsy
350,106309.0,1.0,FTLD-bvFTD
366,106814.0,1.0,FTLD-bvFTD


In [198]:
ftd_autopsyT[ftd_autopsyT['INDDID'].isin(path_id_TAU[tau_aut_over])][['INDDID','BVFTD','ClinicalDx1']].drop_duplicates(subset=['INDDID','BVFTD','ClinicalDx1']).sort_values(by='INDDID').groupby('ClinicalDx1').size().reset_index(name='Counts')

,ClinicalDx1,Counts
0,Corticobasal syndrome,5
1,FTLD-NOS,1
2,FTLD-PPA (PNFA),2
3,FTLD-PPA (Semantic dementia),2
4,FTLD-bvFTD,14
5,Progressive supranuclear palsy,9


In [202]:
ftd_autopsyT[ftd_autopsyT['INDDID'].isin(path_id_TAU[tau_aut_over])][['INDDID','BVFTD','ClinicalDx1']].drop_duplicates(subset=['INDDID','BVFTD','ClinicalDx1']).sort_values(by='INDDID').groupby('BVFTD').size().reset_index(name='Counts')

,BVFTD,Counts
0,0.0,19
1,1.0,14


In [190]:
ftd_autopsyT[ftd_autopsyT['INDDID'].isin(path_id_TDP[tdp_aut_over])][['INDDID','BVFTD','ClinicalDx1']].drop_duplicates(subset=['INDDID','BVFTD','ClinicalDx1']).sort_values(by='INDDID')

,INDDID,BVFTD,ClinicalDx1
268,101045.00,1.0,Amyotrophic lateral sclerosis (Clinically Defi...
1171,101525.00,0.0,Amyotrophic lateral sclerosis (Clinically Prob...
1229,101778.00,0.0,Progressive muscular atrophy
713,102792.00,0.0,Amyotrophic lateral sclerosis (Clinically Defi...
491,104094.00,1.0,FTLD-bvFTD
1172,104613.00,0.0,Corticobasal syndrome
281,104862.00,1.0,FTLD-bvFTD
1067,105247.00,0.0,Amyotrophic lateral sclerosis (Clinically Defi...
1133,105769.00,0.0,FTLD-PPA (Semantic dementia)
686,107519.00,0.0,Amyotrophic lateral sclerosis (Clinically Defi...


In [199]:
ftd_autopsyT[ftd_autopsyT['INDDID'].isin(path_id_TDP[tdp_aut_over])][['INDDID','BVFTD','ClinicalDx1']].drop_duplicates(subset=['INDDID','BVFTD','ClinicalDx1']).sort_values(by='INDDID').groupby('ClinicalDx1').size().reset_index(name='Counts')

,ClinicalDx1,Counts
0,Amyotrophic lateral sclerosis (Clinically Defi...,10
1,Amyotrophic lateral sclerosis (Clinically Prob...,6
2,Corticobasal syndrome,1
3,FTLD-NOS,1
4,FTLD-PPA (PNFA),3
5,FTLD-PPA (Semantic dementia),3
6,FTLD-bvFTD,18
7,Probable Alzheimer's Disease,1
8,Progressive muscular atrophy,2


In [204]:
ftd_autopsyT[ftd_autopsyT['INDDID'].isin(path_id_TDP[tdp_aut_over])][['INDDID','BVFTD','ClinicalDx1']].drop_duplicates(subset=['INDDID','BVFTD','ClinicalDx1']).sort_values(by='INDDID').groupby('BVFTD').size().reset_index(name='Counts')

,BVFTD,Counts
0,0.0,21
1,1.0,24


## MR

In [161]:
tau_aut_over_mr = np.isin(thick_id_TAU, autopsy_id)
print(f"Number of overlap (MR-Tau): {tau_aut_over_mr.sum()}/{len(thick_id_TAU)}")

Number of overlap (MR-Tau): 26/26


In [162]:
tdp_aut_over_mr = np.isin(thick_id_TDP, autopsy_id)
print(f"Number of overlap (MR-Tdp): {tdp_aut_over_mr.sum()}/{len(thick_id_TDP)}")

Number of overlap (MR-Tdp): 30/30


In [207]:
hc_aut_over_mr = np.isin(thick_id_HC, autopsy_id)
print(f"Number of overlap (MR-HC): {hc_aut_over_mr.sum()}/{len(thick_id_HC)}")

Number of overlap (MR-HC): 0/54


## BVFTD

In [192]:
ftd_autopsyT[ftd_autopsyT['INDDID'].isin(thick_id_TAU[tau_aut_over_mr])][['INDDID','BVFTD','ClinicalDx1']].drop_duplicates(subset=['INDDID','BVFTD','ClinicalDx1']).sort_values(by='INDDID')

,INDDID,BVFTD,ClinicalDx1
472,101483.0,1.0,FTLD-bvFTD
450,104093.0,1.0,FTLD-bvFTD
297,104190.0,1.0,FTLD-bvFTD
346,105564.0,1.0,FTLD-bvFTD
350,106309.0,1.0,FTLD-bvFTD
366,106814.0,1.0,FTLD-bvFTD
145,107516.0,1.0,FTLD-bvFTD
129,107677.0,1.0,FTLD-bvFTD
222,108026.0,1.0,FTLD-bvFTD
11,109300.0,1.0,FTLD-bvFTD


In [200]:
ftd_autopsyT[ftd_autopsyT['INDDID'].isin(thick_id_TAU[tau_aut_over_mr])][['INDDID','BVFTD','ClinicalDx1']].drop_duplicates(subset=['INDDID','BVFTD','ClinicalDx1']).sort_values(by='INDDID').groupby('ClinicalDx1').size().reset_index(name='Counts')

,ClinicalDx1,Counts
0,Dementia with Lewy Bodies,1
1,FTLD-bvFTD,25


In [205]:
ftd_autopsyT[ftd_autopsyT['INDDID'].isin(thick_id_TAU[tau_aut_over_mr])][['INDDID','BVFTD','ClinicalDx1']].drop_duplicates(subset=['INDDID','BVFTD','ClinicalDx1']).sort_values(by='INDDID').groupby('BVFTD').size().reset_index(name='Counts')

,BVFTD,Counts
0,1.0,26


In [194]:
ftd_autopsyT[ftd_autopsyT['INDDID'].isin(thick_id_TDP[tdp_aut_over_mr])][['INDDID','BVFTD','ClinicalDx1']].drop_duplicates(subset=['INDDID','BVFTD','ClinicalDx1']).sort_values(by='INDDID')

,INDDID,BVFTD,ClinicalDx1
268,101045.0,1.0,Amyotrophic lateral sclerosis (Clinically Defi...
491,104094.0,1.0,FTLD-bvFTD
281,104862.0,1.0,FTLD-bvFTD
201,105223.0,1.0,FTLD-bvFTD
487,108276.0,1.0,Amyotrophic lateral sclerosis (Clinically Prob...
492,108344.0,1.0,FTLD-bvFTD
296,108790.0,1.0,FTLD-bvFTD
47,109050.0,1.0,FTLD-bvFTD
197,109073.0,1.0,Probable Alzheimer's Disease
63,109476.0,1.0,FTLD-bvFTD


In [201]:
ftd_autopsyT[ftd_autopsyT['INDDID'].isin(thick_id_TDP[tdp_aut_over_mr])][['INDDID','BVFTD','ClinicalDx1']].drop_duplicates(subset=['INDDID','BVFTD','ClinicalDx1']).sort_values(by='INDDID').groupby('ClinicalDx1').size().reset_index(name='Counts')

,ClinicalDx1,Counts
0,Amyotrophic lateral sclerosis (Clinically Defi...,2
1,Amyotrophic lateral sclerosis (Clinically Prob...,2
2,FTLD-NOS,1
3,FTLD-bvFTD,24
4,Probable Alzheimer's Disease,1


In [206]:
ftd_autopsyT[ftd_autopsyT['INDDID'].isin(thick_id_TDP[tdp_aut_over_mr])][['INDDID','BVFTD','ClinicalDx1']].drop_duplicates(subset=['INDDID','BVFTD','ClinicalDx1']).sort_values(by='INDDID').groupby('BVFTD').size().reset_index(name='Counts')

,BVFTD,Counts
0,1.0,30


# Path vs MR

In [131]:
print(f"Number of overlap (Tau - Path vs MR): {np.isin(thick_id_TAU, path_id_TAU).sum()}/{len(thick_id_TAU)}")

Number of overlap (Tau - Path vs MR): 13/26


In [132]:
print(f"Number of overlap (TDP - Path vs MR): {np.isin(thick_id_TDP, path_id_TDP).sum()}/{len(thick_id_TDP)}")

Number of overlap (TDP - Path vs MR): 21/30
